In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'brain-tumor-classification-mri:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F672377%2F1183165%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T172408Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3cd953680a2d4aa0d7981d935ede1fdf8e09af94ba8e1b85b5ad9fbab9fe0279b7d2994b1f14f170d0e0c7989e44ca300d413021fff1e2653bc2c06ad0d3709d51fa0c1efb82d7f1bd6038b051110d24a89fa281dd9c34e5a03f8f691c164a77753d1ebec920652c79bf0dfc8de389d0797584fb9c8cc87f6e1d87d39f683acf80be7951846e0d1f0763d4bea866e86c9cfd5afa56f762862a635adbe6148bbf4759466cdb34796b52066f09cad49db6b7a425edf0f1063042c30b4bd85ee38f5884d10aa987e8884c15e825361ebf8a63ea914ac61e3b46e68d8922cafa9df777eec7b34b7c57d2d6b0d77856c9a2958142c402ff8055c0703c5dfe5b2f1f84'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 91002358 bytes downloaded
Downloaded and uncompressed: brain-tumor-classification-mri
Data source import complete.


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense




# Preprocessing training and testing data

## preprocessing training data

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    '/kaggle/input/brain-tumor-classification-mri/Training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 2870 images belonging to 4 classes.


## Preprocessing testing data

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testing_set = test_datagen.flow_from_directory(
    '/kaggle/input/brain-tumor-classification-mri/Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 394 images belonging to 4 classes.


# Train the dataset

In [ ]:
# resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
cnn = Sequential()
cnn.add(base_model)
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(units=256, activation='relu'))
cnn.add(Dense(units=4, activation='softmax'))

In [ ]:
for layer in base_model.layers[:-8]:
    layer.trainable = True

# Compile cnn with a custom learning rate

In [ ]:

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
cnn.fit(x=training_set, batch_size=32, validation_data=testing_set, epochs=1)

90/90 [==============================] - 2320s 26s/step - loss: 0.3769 - accuracy: 0.8624 - val_loss: 3.5793 - val_accuracy: 0.2665


# Evaluate accuracy on the test set

In [ ]:

accuracy = cnn.evaluate(testing_set)[20]
print("Test Accuracy:", accuracy)

13/13 [==============================] - 76s 6s/step - loss: 3.5763 - accuracy: 0.2665
Test Accuracy: 0.2664974629878998


# Print model summary

In [ ]:

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dense_3 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 24113284 (91.98 MB)
Trainable params: 24060164 (91.78 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________
